In [ ]:
# Health Insurance Premium Prediction Project

## Project Overview
This project aims to predict health insurance premiums using Ordinary Least Squares (OLS) regression with comprehensive feature engineering and selection techniques including:
- Variance Inflation Factor (VIF) analysis for multicollinearity detection
- Correlation analysis for feature selection
- Feature engineering to improve model performance

## Dataset
The dataset contains health insurance information with features like age, sex, BMI, children, smoker status, region, and insurance charges.

## Methodology
1. **Data Exploration and Preprocessing**
2. **Feature Engineering**
3. **Feature Selection using Correlation and VIF**
4. **OLS Model Implementation**
5. **Model Evaluation and Diagnostics**

In [ ]:
# Import necessary libraries for data manipulation, visualization, and modeling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Load the insurance dataset
df = pd.read_csv('insurance.csv')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nSummary Statistics:")
print(df.describe())

## Exploratory Data Analysis (EDA)

In [ ]:
# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

# Check for duplicate records
print(f"\nNumber of duplicate records: {df.duplicated().sum()}")

# Display unique values for categorical columns
categorical_cols = ['sex', 'smoker', 'region']
for col in categorical_cols:
    print(f"\nUnique values in {col}: {df[col].unique()}")
    print(f"Value counts for {col}:")
    print(df[col].value_counts())

In [ ]:
# Create visualizations for data distribution
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Distribution of numerical variables
numerical_cols = ['age', 'bmi', 'children', 'charges']
for i, col in enumerate(numerical_cols):
    row = i // 2
    col_idx = i % 2
    axes[row, col_idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    axes[row, col_idx].set_title(f'Distribution of {col}')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('Frequency')

# Bar plots for categorical variables
axes[0, 2].bar(df['sex'].value_counts().index, df['sex'].value_counts().values)
axes[0, 2].set_title('Distribution of Sex')
axes[0, 2].set_xlabel('Sex')
axes[0, 2].set_ylabel('Count')

axes[1, 2].bar(df['smoker'].value_counts().index, df['smoker'].value_counts().values)
axes[1, 2].set_title('Distribution of Smoker')
axes[1, 2].set_xlabel('Smoker')
axes[1, 2].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# Analyze the target variable (charges) in more detail
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(df['charges'], bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Insurance Charges')
plt.xlabel('Charges')
plt.ylabel('Frequency')

plt.subplot(1, 3, 2)
plt.hist(np.log(df['charges']), bins=50, edgecolor='black', alpha=0.7)
plt.title('Distribution of Log(Charges)')
plt.xlabel('Log(Charges)')
plt.ylabel('Frequency')

plt.subplot(1, 3, 3)
plt.boxplot(df['charges'])
plt.title('Boxplot of Insurance Charges')
plt.ylabel('Charges')

plt.tight_layout()
plt.show()

# Statistical summary of charges
print("Statistical Summary of Insurance Charges:")
print(f"Mean: ${df['charges'].mean():.2f}")
print(f"Median: ${df['charges'].median():.2f}")
print(f"Standard Deviation: ${df['charges'].std():.2f}")
print(f"Minimum: ${df['charges'].min():.2f}")
print(f"Maximum: ${df['charges'].max():.2f}")
print(f"Skewness: {df['charges'].skew():.2f}")
print(f"Kurtosis: {df['charges'].kurtosis():.2f}")

In [ ]:
# Analyze relationships between features and target variable
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Age vs Charges
axes[0, 0].scatter(df['age'], df['charges'], alpha=0.6)
axes[0, 0].set_title('Age vs Insurance Charges')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Charges')

# BMI vs Charges
axes[0, 1].scatter(df['bmi'], df['charges'], alpha=0.6)
axes[0, 1].set_title('BMI vs Insurance Charges')
axes[0, 1].set_xlabel('BMI')
axes[0, 1].set_ylabel('Charges')

# Children vs Charges
axes[0, 2].boxplot([df[df['children'] == i]['charges'].values for i in range(6)], 
                   labels=range(6))
axes[0, 2].set_title('Children vs Insurance Charges')
axes[0, 2].set_xlabel('Number of Children')
axes[0, 2].set_ylabel('Charges')

# Sex vs Charges
sns.boxplot(data=df, x='sex', y='charges', ax=axes[1, 0])
axes[1, 0].set_title('Sex vs Insurance Charges')

# Smoker vs Charges
sns.boxplot(data=df, x='smoker', y='charges', ax=axes[1, 1])
axes[1, 1].set_title('Smoker vs Insurance Charges')

# Region vs Charges
sns.boxplot(data=df, x='region', y='charges', ax=axes[1, 2])
axes[1, 2].set_title('Region vs Insurance Charges')
axes[1, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Feature Engineering